In [23]:
import tensorflow as tf
import torch
from transformers import BertTokenizer
import pickle
import os
import keras_bert
import zipfile
from sklearn.datasets import load_files
import chardet
from keras.preprocessing.sequence import pad_sequences

In [24]:
def standardize_to_utf8(encoding):
    """
    standardize to utf-8 if necessary.
    NOTE: mainly used to use utf-8 if ASCII is detected, as
    BERT performance suffers otherwise.
    """
    encoding = 'utf-8' if encoding.lower() in ['ascii', 'utf8', 'utf-8'] else encoding
    return encoding

In [25]:
train_test_names = ["train", "test"]
datadir = "dataset_up"
classes = ["class0", "class1"]
train_str = train_test_names[0]
train_b = load_files(os.path.join(datadir, train_str), shuffle=False, categories=classes)
test_str = train_test_names[1]
test_b = load_files(os.path.join(datadir,  test_str), shuffle=False, categories=classes)
x_train = train_b.data
y_train = train_b.target
x_test = test_b.data
y_test = test_b.target

In [26]:
lst = [chardet.detect(doc)['encoding'] for doc in x_train[:32]]
encoding = max(set(lst), key=lst.count)
encoding = standardize_to_utf8(encoding)
x_train = [x.decode(encoding) for x in x_train]
x_test = [x.decode(encoding) for x in x_test]

In [27]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True)

In [6]:
def get_token_ids(x_train, x_test):
    
    token_tr = []
    token_tst = []
    count = 0
#     for sent in x_train :
#         tokens = tokenizer.encode(sent, add_special_tokens = True, max_length=512)
#         token_tr.append(tokens)
#         count+=1
#         if(count%1000==0):
#             print(count)
    
    for sent1 in x_test :
        tokens1 = tokenizer.encode(sent1, add_special_tokens = True, max_length=512)
        token_tst.append(tokens1)
        count+=1
        if(count%1000==0):
            print(count)
            
    return token_tr, token_tst 

In [7]:
xtr_token, xtst_token = get_token_ids(x_train, x_test)

1000
2000
3000
4000
5000
6000


In [8]:
xtr_token = pad_sequences(xtr_token, maxlen=512, dtype="long", 
                          value=0, truncating="post", padding="post")
xtst_token = pad_sequences(xtst_token, maxlen=512, dtype="long", 
                          value=0, truncating="post", padding="post")

In [9]:
attention_mask_tr = []
attention_mask_tst = []
for sent in xtr_token:
    att_mask = [int(token_id > 0) for token_id in sent]
    attention_mask_tr.append(att_mask)

In [10]:
for sent in xtst_token:
    att_mask = [int(token_id > 0) for token_id in sent]
    attention_mask_tst.append(att_mask)

In [11]:
train_input = torch.tensor(xtr_token)
test_input = torch.tensor(xtst_token)

train_label = torch.tensor(y_train)
test_label = torch.tensor(y_test)

train_mask = torch.tensor(attention_mask_tr)
test_mask = torch.tensor(attention_mask_tst)

In [18]:
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler
batch_size = 32

# Train_data = TensorDataset(train_input, train_mask, train_label)
# Train_sampler = SequentialSampler(Train_data)
# Train_dataloader = DataLoader(Train_data, sampler=Train_sampler, batch_size=batch_size)

# Create the DataLoader for our validation set.
Test_data = TensorDataset(test_input, test_mask, test_label)
Test_sampler = SequentialSampler(Test_data)
Test_dataloader = DataLoader(Test_data, sampler=Test_sampler, batch_size=batch_size)

In [13]:
import torch.nn as nn
from transformers import BertModel

class ContextEmbeddings(nn.Module):

    def __init__(self, freeze_bert = True):
        super(ContextEmbeddings, self).__init__()
        self.bert_layer = BertModel.from_pretrained('bert-base-uncased')
        
        if freeze_bert:
            for p in self.bert_layer.parameters():
                p.requires_grad = False
                
        self.embedding_layer = nn.Linear(768, 256)
        self.cls_layer = nn.Linear(256, 1)

    def forward(self, seq, attn_masks):
        '''
        Inputs:
            -seq : Tensor of shape [B, T] containing token ids of sequences
            -attn_masks : Tensor of shape [B, T] containing attention masks to be used to avoid contibution of PAD tokens
        '''

        #Feeding the input to BERT model to obtain contextualized representations
        cont_reps, _ = self.bert_layer(seq, attention_mask = attn_masks)

        #Obtaining the representation of [CLS] head
        embedding_rep = cont_reps[:, 0]
        embeds = self.embedding_layer(embedding_rep)
#         cls_rep = embeds[:,0]
        #Feeding cls_rep to the classifier layer
        logits = self.cls_layer(embeds)

        return logits

In [14]:
model = ContextEmbeddings(freeze_bert=  True)

In [15]:
model.load_state_dict(torch.load("models/model_state_dict"))

<All keys matched successfully>

In [16]:
model.eval()

ContextEmbeddings(
  (bert_layer): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_a

In [22]:
output_train = []
for it, (seq, attn_masks, labels) in enumerate(Train_dataloader):
    bert_out, _ = model.bert_layer.forward(seq, attn_masks)
#     embeds = model.embedding_layer.forward(bert_out[:,0])
    
    output_train.append(bert_out[:,0])
    print(it)

NameError: name 'Train_dataloader' is not defined

In [ ]:
pickle.dump(output_train, open("models/output_train.pkl","wb"))
print("Output_train_pickled...YAYYYYY")

In [28]:
output_test = []
for it, (seq, attn_masks, labels) in enumerate(Test_dataloader) :
    bert_out, _ = model.bert_layer.forward(seq, attn_masks)
#     embeds = model.embedding_layer.forward(bert_out[:,0])
    output_test.append(bert_out[:,0])
    print(it)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201


In [29]:
pickle.dump(output_test, open("models/output_test_only_bert.pkl","wb"))
print("Output_test_pickled...YAYYYYY")

Output_test_pickled...YAYYYYY


In [21]:
output_train = pickle.load(open("models/output_train.pkl", "rb"))
output_test = pickle.load(open("models/output_test_actual.pkl", "rb"))

In [3]:
output_train[0]

tensor([[ 0.1531,  0.6999,  0.3711,  ..., -0.0465,  0.4393,  0.0193],
        [ 0.2463,  0.2860,  0.5043,  ..., -0.2459,  0.4536, -0.0490],
        [ 0.1231,  0.7580,  0.5951,  ...,  0.0421,  0.3562,  0.0673],
        ...,
        [-0.0497,  0.7057,  0.5475,  ..., -0.2927,  0.3791, -0.1126],
        [-0.2692,  0.7150,  0.4441,  ..., -0.0486,  0.5564, -0.1518],
        [ 0.3656,  0.5258,  0.2436,  ..., -0.1255,  0.4075, -0.0252]],
       requires_grad=True)

In [5]:
out_tr_numpy = []
for x in output_train :
    nmp = x.detach().numpy()
    out_tr_numpy.extend(list(nmp))

In [13]:
out_ts_numpy = []
for y in output_test :
    nmp = y.detach().numpy()
    out_ts_numpy.extend(list(nmp))

In [14]:
len(out_ts_numpy)

25788

In [12]:
len(output_test)

806